In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pylab as pl
import os
import plotly
from fiona.crs import from_epsg
from geopandas.tools import sjoin
from IPython.display import display, clear_output
import time

% pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
# REad in Schmelt form data
schmelt = pd.read_csv('data/Schmelt_bg.csv', index_col=0)

# Ensure formated as numerical
schmelt[['1996', '1997', '1998', '1999', '2000', '2001',
         '2002', '2003', '2004', '2005', '2006', '2007', 
         '2008', '2009', '2010','2011', '2012', '2013', 
         '2014', '2015', '2016']] = schmelt.iloc[:,2:].apply(pd.to_numeric, errors='coerce')

# Drop completely NA values
schmelt.dropna(axis=0,subset=['1996', '1997', '1998', '1999', '2000', '2001',
                              '2002', '2003', '2004', '2005', '2006', '2007', 
                              '2008', '2009', '2010','2011', '2012', '2013', 
                              '2014', '2015', '2016'], how='all', inplace=True)

# Preview Data
schmelt.head()

,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,93.98,93.56,94.45,93.49,93.00,94.86,94.81,93.87,95.12,95.62
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,93.16,94.99,94.87,95.46,93.83,94.68,94.86,95.12,95.38,95.55
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,94.57,94.37,95.75,95.55,95.14,95.57,95.35,95.26,95.74,96.32
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,94.00,94.81,93.66,95.50,95.65,96.35,95.85,95.16,95.67,96.50
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,94.02,94.43,95.38,94.12,95.74,95.81,96.40,94.99,95.64,95.79


In [5]:
schmelt.tail()

,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
291396,X966,SCHHIGRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
291397,X966,SCHLEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN
291398,X966,SCHLEV5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN
291399,X966,SCHLOGRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
291401,X966,SPECHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


# 1996 to 2016 data set

In [29]:
# Create Copy of Original Data
schmelt96_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt96_16['MinYear'] = np.nan
schmelt96_16['MaxYear'] = np.nan
schmelt96_16['trend96_16'] = np.nan

# Determine Max Years
for cols in schmelt96_16.loc[:,'1996':'2016'].columns:
    schmelt96_16['MaxYear'][schmelt96_16.index[schmelt96_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt96_16.loc[:,'2016':'1996':-1].columns:
    schmelt96_16['MinYear'][schmelt96_16.index[schmelt96_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt96_16['mean96_16'] = schmelt96_16.loc[:,'1996':'2016'].T.mean()
schmelt96_16['std96_16'] = schmelt96_16.loc[:,'1996':'2016'].T.std()

# Calculate Trend
for cols in schmelt96_16.loc[:,'1996':'2016'].columns:
    for cols2 in schmelt96_16.loc[:,'2016':'1996':-1].columns:
        schmelt96_16['trend96_16'][schmelt96_16['trend96_16'].isnull()] = (schmelt96_16[cols2][schmelt96_16['trend96_16'].isnull()] - 
                                                                           schmelt96_16[cols][schmelt96_16['trend96_16'].isnull()]) / (float(cols2) - float(cols)) 

# Create range feature for range of years there are data.
schmelt96_16['range'] = pd.to_numeric(schmelt96_16['MaxYear']) - pd.to_numeric(schmelt96_16['MinYear'])

# Pivot each feature to convert back to schma format
trends = schmelt96_16.pivot(index='BNLONG',columns='variable', values='trend96_16')
means = schmelt96_16.pivot(index='BNLONG',columns='variable', values='mean96_16')
stds = schmelt96_16.pivot(index='BNLONG',columns='variable', values='std96_16')
ranges = schmelt96_16.pivot(index='BNLONG',columns='variable', values='range')

# Rename columns
trends.columns = trends.columns + ' trend96_16'
means.columns = means.columns + ' mean96_16'
stds.columns = stds.columns + ' std96_16'
ranges.columns = ranges.columns + ' range96_16'

# Merge data sets back together
schmelt96_16_2 = pd.concat([trends, means, stds, ranges], axis=1)

# Export as csv
schmelt96_16_2.to_csv('data/schmelt96-16.csv')

# Preview 
schmelt96_16_2.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ATTPCTG01 trend96_16,ATTPCTG02 trend96_16,ATTPCTG03 trend96_16,ATTPCTG04 trend96_16,ATTPCTG05 trend96_16,ATTPCTG06 trend96_16,ATTPCTG07 trend96_16,ATTPCTG08 trend96_16,ATTPCTG09 trend96_16,ATTPCTG10 trend96_16,...,STCELAPCTL34G07 range96_16,STCELAPCTL34G08 range96_16,STCELAPCTL34TOT range96_16,STCMTHPCTL34G03 range96_16,STCMTHPCTL34G04 range96_16,STCMTHPCTL34G05 range96_16,STCMTHPCTL34G06 range96_16,STCMTHPCTL34G07 range96_16,STCMTHPCTL34G08 range96_16,STCMTHPCTL34TOT range96_16
BNLONG,,,,,,,,,,,,,,,,,,,,,
K001,0.343,0.2110,0.1945,0.207,0.1375,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,17.0,17.0,17.0,NaN,NaN,NaN,17.0
K002,NaN,NaN,NaN,NaN,NaN,0.224500,0.3415,0.351,-16.28,NaN,...,17.0,17.0,17.0,NaN,NaN,NaN,17.0,17.0,17.0,17.0
K003,0.267,0.0255,0.0830,0.170,0.2225,0.660588,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,17.0,17.0,17.0,5.0,NaN,NaN,17.0
K004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.0,12.0,13.0,13.0,0.0,NaN,NaN,13.0
K005,0.368,0.2385,0.2580,0.288,0.4385,-7.097692,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,17.0,17.0,17.0,5.0,NaN,NaN,17.0


# 2013 to 2016 data set - 4 Year Data Set

In [30]:
# Create Copy of Original Data
schmelt13_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt13_16['MinYear'] = np.nan
schmelt13_16['MaxYear'] = np.nan
schmelt13_16['trend13_16'] = np.nan

# Determine Max Years
for cols in schmelt13_16.loc[:,'2013':'2016'].columns:
    schmelt13_16['MaxYear'][schmelt13_16.index[schmelt13_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt13_16.loc[:,'2016':'2013':-1].columns:
    schmelt13_16['MinYear'][schmelt13_16.index[schmelt13_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt13_16['mean13_16'] = schmelt13_16.loc[:,'2013':'2016'].T.mean()
schmelt13_16['std13_16'] = schmelt13_16.loc[:,'2013':'2016'].T.std()

# Calculate Trend
for cols in schmelt13_16.loc[:,'2013':'2016'].columns:
    for cols2 in schmelt13_16.loc[:,'2016':'2013':-1].columns:
        schmelt13_16['trend13_16'][schmelt13_16['trend13_16'].isnull()] = (schmelt13_16[cols2][schmelt13_16['trend13_16'].isnull()] - 
                                                                           schmelt13_16[cols][schmelt13_16['trend13_16'].isnull()]) / (float(cols2) - float(cols))

# Create range feature for range of years there are data.
schmelt13_16['range'] = pd.to_numeric(schmelt13_16['MaxYear']) - pd.to_numeric(schmelt13_16['MinYear'])

# Pivot each feature to convert back to schma format
trends = schmelt13_16.pivot(index='BNLONG',columns='variable', values='trend13_16')
means = schmelt13_16.pivot(index='BNLONG',columns='variable', values='mean13_16')
stds = schmelt13_16.pivot(index='BNLONG',columns='variable', values='std13_16')
ranges = schmelt13_16.pivot(index='BNLONG',columns='variable', values='range')

# Rename columns
trends.columns = trends.columns + ' trend13_16'
means.columns = means.columns + ' mean13_16'
stds.columns = stds.columns + ' std13_16'
ranges.columns = ranges.columns + ' range13_16'

# Merge data sets back together
schmelt13_16_2 = pd.concat([trends, means, stds, ranges], axis=1)

# Export as csv
schmelt13_16_2.to_csv('data/schmelt13-16.csv')        
        
# Preview 
schmelt13_16.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,BNLONG,variable,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,MinYear,MaxYear,trend13_16,mean13_16,std13_16,range
0,K001,ATTPCTG01,88.76,92.53,90.76,91.04,91.62,91.57,91.88,92.76,...,94.81,93.87,95.12,95.62,2013,2016,0.270000,94.8550,0.736591,3.0
1,K001,ATTPCTG02,91.33,91.37,92.66,92.00,92.26,93.87,92.71,93.02,...,94.86,95.12,95.38,95.55,2013,2016,0.230000,95.2275,0.302145,3.0
2,K001,ATTPCTG03,92.43,93.11,92.36,93.55,92.90,92.77,94.14,93.56,...,95.35,95.26,95.74,96.32,2013,2016,0.323333,95.6675,0.482312,3.0
3,K001,ATTPCTG04,92.36,93.75,92.44,91.88,93.99,93.17,92.38,94.30,...,95.85,95.16,95.67,96.50,2013,2016,0.216667,95.7950,0.553444,3.0
4,K001,ATTPCTG05,93.04,92.92,92.61,93.20,92.07,94.12,93.25,93.45,...,96.40,94.99,95.64,95.79,2013,2016,-0.203333,95.7050,0.578993,3.0


# 2009 to 2016 data set - 8 Year Data Set

In [31]:
# Create Copy of Original Data
schmelt09_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt09_16['MinYear'] = np.nan
schmelt09_16['MaxYear'] = np.nan
schmelt09_16['trend09_16'] = np.nan

# Determine Max Years
for cols in schmelt09_16.loc[:,'2009':'2016'].columns:
    schmelt09_16['MaxYear'][schmelt09_16.index[schmelt09_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt09_16.loc[:,'2016':'2009':-1].columns:
    schmelt09_16['MinYear'][schmelt09_16.index[schmelt09_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt09_16['mean09_16'] = schmelt09_16.loc[:,'2009':'2016'].T.mean()
schmelt09_16['std09_16'] = schmelt09_16.loc[:,'2009':'2016'].T.std()

# Calculate Trend
for cols in schmelt09_16.loc[:,'2009':'2016'].columns:
    for cols2 in schmelt09_16.loc[:,'2016':'2009':-1].columns:
        schmelt09_16['trend09_16'][schmelt09_16['trend09_16'].isnull()] = (schmelt09_16[cols2][schmelt09_16['trend09_16'].isnull()] - 
                                                                           schmelt09_16[cols][schmelt09_16['trend09_16'].isnull()]) / (float(cols2) - float(cols))

# Create range feature for range of years there are data.
schmelt09_16['range'] = pd.to_numeric(schmelt09_16['MaxYear']) - pd.to_numeric(schmelt09_16['MinYear'])

# Pivot each feature to convert back to schma format
trends = schmelt09_16.pivot(index='BNLONG',columns='variable', values='trend09_16')
means = schmelt09_16.pivot(index='BNLONG',columns='variable', values='mean09_16')
stds = schmelt09_16.pivot(index='BNLONG',columns='variable', values='std09_16')
ranges = schmelt09_16.pivot(index='BNLONG',columns='variable', values='range')

# Rename columns
trends.columns = trends.columns + ' trend09_16'
means.columns = means.columns + ' mean09_16'
stds.columns = stds.columns + ' std09_16'
ranges.columns = ranges.columns + ' range09_16'

# Merge data sets back together
schmelt09_16_2 = pd.concat([trends, means, stds, ranges], axis=1)

# Export as csv
schmelt09_16_2.to_csv('data/schmelt09-16.csv')        
        
# Preview 
schmelt09_16_2.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ATTPCTG01 trend09_16,ATTPCTG02 trend09_16,ATTPCTG03 trend09_16,ATTPCTG04 trend09_16,ATTPCTG05 trend09_16,ATTPCTG06 trend09_16,ATTPCTG07 trend09_16,ATTPCTG08 trend09_16,ATTPCTG09 trend09_16,ATTPCTG10 trend09_16,...,STCELAPCTL34G07 range09_16,STCELAPCTL34G08 range09_16,STCELAPCTL34TOT range09_16,STCMTHPCTL34G03 range09_16,STCMTHPCTL34G04 range09_16,STCMTHPCTL34G05 range09_16,STCMTHPCTL34G06 range09_16,STCMTHPCTL34G07 range09_16,STCMTHPCTL34G08 range09_16,STCMTHPCTL34TOT range09_16
BNLONG,,,,,,,,,,,,,,,,,,,,,
K001,0.167143,0.097143,0.081429,0.405714,0.058571,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,7.0,7.0,7.0,NaN,NaN,NaN,7.0
K002,NaN,NaN,NaN,NaN,NaN,0.288571,0.435714,0.315714,NaN,NaN,...,7.0,7.0,7.0,NaN,NaN,NaN,7.0,7.0,7.0,7.0
K003,-0.075714,-0.288571,-0.027143,0.044286,0.185714,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,7.0,7.0,7.0,NaN,NaN,NaN,7.0
K004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.0,3.0,3.0,NaN,NaN,NaN,3.0
K005,0.287143,0.315714,-0.214286,0.274286,0.381429,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,7.0,7.0,7.0,NaN,NaN,NaN,7.0


# 2005 to 2016 data set - 12 Year Data Set

In [34]:
# Create Copy of Original Data
schmelt05_16 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt05_16['MinYear'] = np.nan
schmelt05_16['MaxYear'] = np.nan
schmelt05_16['trend05_16'] = np.nan

# Determine Max Years
for cols in schmelt05_16.loc[:,'2005':'2016'].columns:
    schmelt05_16['MaxYear'][schmelt05_16.index[schmelt05_16[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt05_16.loc[:,'2016':'2005':-1].columns:
    schmelt05_16['MinYear'][schmelt05_16.index[schmelt05_16[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt05_16['mean05_16'] = schmelt05_16.loc[:,'2005':'2016'].T.mean()
schmelt05_16['std05_16'] = schmelt05_16.loc[:,'2005':'2016'].T.std()

# Calculate Trend
for cols in schmelt05_16.loc[:,'2005':'2016'].columns:
    for cols2 in schmelt05_16.loc[:,'2016':'2005':-1].columns:
        schmelt05_16['trend05_16'][schmelt05_16['trend05_16'].isnull()] = (schmelt05_16[cols2][schmelt05_16['trend05_16'].isnull()] - 
                                                                           schmelt05_16[cols][schmelt05_16['trend05_16'].isnull()]) / (float(cols2) - float(cols))

        
# Create range feature for range of years there are data.
schmelt05_16['range'] = pd.to_numeric(schmelt05_16['MaxYear']) - pd.to_numeric(schmelt05_16['MinYear'])

# Pivot each feature to convert back to schma format
trends = schmelt05_16.pivot(index='BNLONG',columns='variable', values='trend05_16')
means = schmelt05_16.pivot(index='BNLONG',columns='variable', values='mean05_16')
stds = schmelt05_16.pivot(index='BNLONG',columns='variable', values='std05_16')
ranges = schmelt05_16.pivot(index='BNLONG',columns='variable', values='range')

# Rename columns
trends.columns = trends.columns + ' trend05_16'
means.columns = means.columns + ' mean05_16'
stds.columns = stds.columns + ' std05_16'
ranges.columns = ranges.columns + ' range05_16'

# Merge data sets back together
schmelt05_16_2 = pd.concat([trends, means, stds, ranges], axis=1)

# Export as csv
schmelt05_16_2.to_csv('data/schmelt05-16.csv')

# Preview 
schmelt05_16_2.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ATTPCTG01 trend05_16,ATTPCTG02 trend05_16,ATTPCTG03 trend05_16,ATTPCTG04 trend05_16,ATTPCTG05 trend05_16,ATTPCTG06 trend05_16,ATTPCTG07 trend05_16,ATTPCTG08 trend05_16,ATTPCTG09 trend05_16,ATTPCTG10 trend05_16,...,STCELAPCTL34G07 range05_16,STCELAPCTL34G08 range05_16,STCELAPCTL34TOT range05_16,STCMTHPCTL34G03 range05_16,STCMTHPCTL34G04 range05_16,STCMTHPCTL34G05 range05_16,STCMTHPCTL34G06 range05_16,STCMTHPCTL34G07 range05_16,STCMTHPCTL34G08 range05_16,STCMTHPCTL34TOT range05_16
BNLONG,,,,,,,,,,,,,,,,,,,,,
K001,0.220909,0.189091,0.204545,0.250909,0.177273,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.0,11.0,11.0,11.0,NaN,NaN,NaN,11.0
K002,NaN,NaN,NaN,NaN,NaN,0.3200,0.530909,0.168182,-16.28,NaN,...,11.0,11.0,11.0,NaN,NaN,NaN,11.0,11.0,11.0,11.0
K003,0.290909,0.019091,0.089091,0.065455,0.181818,10.0000,NaN,NaN,NaN,NaN,...,NaN,NaN,11.0,11.0,11.0,11.0,NaN,NaN,NaN,11.0
K004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,6.0,7.0,7.0,NaN,NaN,NaN,7.0
K005,0.610000,0.410000,0.164545,0.283636,0.459091,-1.0875,NaN,NaN,NaN,NaN,...,NaN,NaN,11.0,11.0,11.0,11.0,NaN,NaN,NaN,11.0


# 2011 to 2015 data set - Growth Data Set

In [35]:
# Create Copy of Original Data
schmelt11_15 = schmelt.copy()

# Determine min and max year and trend for each row within year range
schmelt11_15['MinYear'] = np.nan
schmelt11_15['MaxYear'] = np.nan
schmelt11_15['trend11_15'] = np.nan

# Determine Max Years
for cols in schmelt11_15.loc[:,'2011':'2015'].columns:
    schmelt11_15['MaxYear'][schmelt11_15.index[schmelt11_15[cols].isnull()==False]] = cols
    
# Determine Min Years
for cols in schmelt11_15.loc[:,'2015':'2011':-1].columns:
    schmelt11_15['MinYear'][schmelt11_15.index[schmelt11_15[cols].isnull()==False]] = cols


# CReate Columns for Mean, Standard Deviation
schmelt11_15['mean11_15'] = schmelt11_15.loc[:,'2011':'2015'].T.mean()
schmelt11_15['std11_15'] = schmelt11_15.loc[:,'2011':'2015'].T.std()

# Calculate Trend
for cols in schmelt11_15.loc[:,'2011':'2015'].columns:
    for cols2 in schmelt11_15.loc[:,'2015':'2011':-1].columns:
        schmelt11_15['trend11_15'][schmelt11_15['trend11_15'].isnull()] = (schmelt11_15[cols2][schmelt11_15['trend11_15'].isnull()] - 
                                                                           schmelt11_15[cols][schmelt11_15['trend11_15'].isnull()]) / (float(cols2) - float(cols))

# Create range feature for range of years there are data.
schmelt11_15['range'] = pd.to_numeric(schmelt11_15['MaxYear']) - pd.to_numeric(schmelt11_15['MinYear'])

# Pivot each feature to convert back to schma format
trends = schmelt11_15.pivot(index='BNLONG',columns='variable', values='trend11_15')
means = schmelt11_15.pivot(index='BNLONG',columns='variable', values='mean11_15')
stds = schmelt11_15.pivot(index='BNLONG',columns='variable', values='std11_15')
ranges = schmelt11_15.pivot(index='BNLONG',columns='variable', values='range')

# Rename columns
trends.columns = trends.columns + ' trend11_15'
means.columns = means.columns + ' mean11_15'
stds.columns = stds.columns + ' std11_15'
ranges.columns = ranges.columns + ' range11_15'

# Merge data sets back together
schmelt11_15_2 = pd.concat([trends, means, stds, ranges], axis=1)

# Export as csv
schmelt11_15_2.to_csv('data/schmelt11-15.csv')
        
# Preview 
schmelt11_15_2.head()

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ATTPCTG01 trend11_15,ATTPCTG02 trend11_15,ATTPCTG03 trend11_15,ATTPCTG04 trend11_15,ATTPCTG05 trend11_15,ATTPCTG06 trend11_15,ATTPCTG07 trend11_15,ATTPCTG08 trend11_15,ATTPCTG09 trend11_15,ATTPCTG10 trend11_15,...,STCELAPCTL34G07 range11_15,STCELAPCTL34G08 range11_15,STCELAPCTL34TOT range11_15,STCMTHPCTL34G03 range11_15,STCMTHPCTL34G04 range11_15,STCMTHPCTL34G05 range11_15,STCMTHPCTL34G06 range11_15,STCMTHPCTL34G07 range11_15,STCMTHPCTL34G08 range11_15,STCMTHPCTL34TOT range11_15
BNLONG,,,,,,,,,,,,,,,,,,,,,
K001,0.5300,0.3875,0.1500,0.005,-0.025,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,4.0,4.0,4.0,NaN,NaN,NaN,4.0
K002,NaN,NaN,NaN,NaN,NaN,0.56,-0.2875,0.5225,NaN,NaN,...,4.0,4.0,4.0,NaN,NaN,NaN,4.0,4.0,4.0,4.0
K003,-0.6625,1.1250,-0.5325,0.280,-0.195,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,4.0,4.0,4.0,NaN,NaN,NaN,4.0
K004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,0.0,1.0,0.0,NaN,NaN,NaN,1.0
K005,0.2850,-0.3075,-0.3700,-0.700,-0.550,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,4.0,4.0,4.0,NaN,NaN,NaN,4.0
